# 补充知识点2：新闻爬取 - 百度新闻爬虫

百度新闻爬虫的相关知识点在笔者的上一本书《Python金融大数据挖掘与分析全流程详解》（https://item.jd.com/12568751.html?dist=jd）

第三章“金融数据挖掘案例实战1”中有详细的讲解，这里结合了本书第二章pandas库的相关知识点将数据进行整合，并导出成Excel文件方便使用。

因为核心知识点在上一本书已经描述的较为详细，所以这里就不在赘述，通过如下代码可以爬取百度新闻多页多个关键词。

In [2]:
import requests
import re
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'}

def baidu(keyword, page):  # 定义函数，方便之后批量调用
    num = (page - 1) * 10
    url = 'https://www.baidu.com/s?tn=news&rtt=4&bsst=1&cl=2&wd=' + keyword + '&pn=' + str(num)
    res = requests.get(url, headers=headers).text  # 通过requests库爬虫
    
    # 正则提取信息 - 网址和标题
    p_href = '<h3 class="news-title_1YtI1"><a href="(.*?)"'
    href = re.findall(p_href, res)
    p_title = '<h3 class="news-title_1YtI1">.*?>(.*?)</a>'
    title = re.findall(p_title, res, re.S)
    # 正则提取信息 - 日期和来源
    p_date = '<span class="c-color-gray2 c-font-normal">(.*?)</span>'
    date = re.findall(p_date, res)
    p_source = '<span class="c-color-gray c-font-normal c-gap-right">(.*?)</span>'
    source = re.findall(p_source, res)

    # 数据清洗
    for i in range(len(title)):  # range(len(title)),这里因为知道len(title) = 10，所以也可以写成for i in range(10)
        title[i] = re.sub('<.*?>', '', title[i])  # 核心，用re.sub()函数来替换不重要的内容
        print(str(i + 1) + '.' + title[i] + ' ' + source[i] + ' ' + date[i])  # 这个是纯字符串拼接
        print(href[i])
    
    # 通过2.2.1节字典生成二维DataFrame表格   
    result = pd.DataFrame({'关键词': keyword, '标题': title, '网址': href, '来源': source, '日期': date})
    return result 

In [3]:
# 通过pandas库将数据进行整合并导出为Excel
import pandas as pd  
df = pd.DataFrame()
    
keywords = ['华能信托', '人工智能', '科技', '体育', 'Python', '娱乐', '文化', '阿里巴巴', '腾讯', '京东']
for keyword in keywords:
    for i in range(10):  # 循环10遍，获取10页的信息
        result = baidu(keyword, i+1)
        df = df.append(result)  # 通过append()函数添加每条信息到df中
        print(keyword + '第' + str(i+1) + '页爬取成功')

df.to_excel('新闻_new.xlsx')  # 在代码所在文件夹生成EXCEL文件

IndexError: list index out of range

In [2]:
df.head()

,关键词,标题,网址,来源,日期
0,华能信托,盈峰环境(000967.SZ):华能信托卖出586.27万张盈峰转债,https://baijiahao.baidu.com/s?id=1685054571206...,智通财经,2020年12月3日 19:01
1,华能信托,盈峰环境(000967.SZ):华能信托卖出盈峰转债合计586.2664万张,http://sc.stock.cnfol.com/ggzixun/20201203/285...,中金在线,2020年12月3日 18:37
2,华能信托,盈峰环境:华能信托于2日购买盈峰转债合计672.4万张,https://baijiahao.baidu.com/s?id=1685027676162...,金融界,2020年12月3日 11:50
3,华能信托,信托日报(12月3日),https://baijiahao.baidu.com/s?id=1685018534787...,财联社,2020年12月3日 09:04
4,华能信托,...华润信托、中粮信托、五矿信托、华能信托、光大信托、华鑫信托...,https://www.otcbeta.com/kx/35b2e0bc9e546cb10ef...,市商网,2020年12月3日 06:19
